In [1]:
import os
from xml.dom import minidom
from xml.dom.minidom import Document
import numpy as np
import torch
from tqdm import tqdm

In [2]:
def listd(folders):
    return [ folder + '/' + f for folder in folders for f in os.listdir(folder)]

In [3]:
beetle = {
  'train': listd([ 
    'datasets/semeval2013-Task7-2and3way/training/3way/beetle' 
  ]),
  'unseen_answers': listd([ 
    'datasets/semeval2013-Task7-2and3way/test/3way/beetle/test-unseen-answers'
  ]),
  'unseen_questions': listd([ 
    'datasets/semeval2013-Task7-2and3way/test/3way/beetle/test-unseen-questions'
  ])
}

sciEntsBank = {
  'train': listd([ 
    'datasets/semeval2013-Task7-2and3way/training/3way/sciEntsBank' 
  ]),
  'unseen_answers': listd([
    'datasets/semeval2013-Task7-2and3way/test/3way/sciEntsBank/test-unseen-answers'
  ]),
  'unseen_questions': listd([
    'datasets/semeval2013-Task7-2and3way/test/3way/sciEntsBank/test-unseen-questions'
  ]),
  'unseen_domains': listd([
    'datasets/semeval2013-Task7-2and3way/test/3way/sciEntsBank/test-unseen-domains'
  ]),
}

def union():
    u = {}
    for key, value in sciEntsBank.items():
        u[key] = value
    
    for key, value in beetle.items():
        u[key] += value
    
    return u

union = union()

In [ ]:
def parse(files, translator):
    doc = Document()
    root = doc.createElement("exercises")
    for file in tqdm(files):
        exercise = doc.createElement('exercise')
        f = minidom.parse(file)
        
        question = doc.createElement('question')
        question.appendChild(
            doc.createTextNode(
                translator(
                    f.getElementsByTagName('questionText')[0].firstChild.data
                )
            )
        )
        exercise.appendChild(question)
        
        references = doc.createElement('references')
        for r in map(lambda a: a.firstChild.data, f.getElementsByTagName('referenceAnswer')):
            reference = doc.createElement('reference')
            reference.appendChild(doc.createTextNode(translator(r)))
            references.appendChild(reference)

        answers = doc.createElement('answers')
        ansL = f.getElementsByTagName('studentAnswer')
        for a, v in map(lambda a: (a.firstChild.data, a.attributes['accuracy'].value), ansL):
            answer = doc.createElement('answer')
            answer.setAttribute('accuracy', v)
            answer.appendChild(doc.createTextNode(translator(a)))
            answers.appendChild(answer)

        exercise.appendChild(references)
        exercise.appendChild(answers)
        root.appendChild(exercise)
    doc.appendChild(root)
    
    return doc

In [ ]:
def process(dataset, name, translator):
    for key, value in dataset.items():
        doc = parse(value, translator)
        
        if not os.path.isdir('datasets/' + name):
            os.mkdir('datasets/' + name)
        
        doc.writexml(open('datasets/' + name + '/' + key + '.xml', 'w'),
               indent="  ",
               addindent="  ",
               newl='\n')
        doc.unlink()

In [8]:
en2de = torch.hub.load('pytorch/fairseq', 'transformer.wmt19.en-de.single_model', tokenizer='moses', bpe='fastbpe')

Using cache found in C:\Users\Leon/.cache\torch\hub\pytorch_fairseq_master


OSError: [Errno 22] Invalid argument

In [ ]:
en2en = lambda v: v

In [ ]:
process(beetle, 'beetle_en', en2en)
process(sciEntsBank, 'sciEntsBank_en', en2en)
process(union, 'union_en', en2en)
process(beetle, 'beetle_de', en2de.translate)
process(sciEntsBank, 'sciEntsBank_de', en2de.translate)
process(union, 'union_de', en2de.translate)